# Proyecto 1 - Grupo G - Comparador de precios de vuelos navideños: encuentra las mejores ofertas para tus vacaciones de navidad

#### Autores: Daniel Muñoz, José Dos Reis y Pamela J. Colman Vega

## Importación de librerías

In [1]:
#Librerías para arrays y data frames
import numpy as np
import pandas as pd

#Librerías para datos de fechas, horas, tiempo
import time
from time import sleep
from datetime import datetime
from datetime import timedelta
import re

#Librerías para el Web Scraping
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import locale

#Librerías para cargar API_KEYS y demás funciones del sistema operativo
import os
from dotenv import load_dotenv
import tqdm

#Librerías para gestión de archivos o ficheros
import pickle
import json

#Librerías para hacer gráficas y plots
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import folium

## Extracción de los datos: web scraping de eDreams y Kayak

#### Los datos son obtenidos mediante scraping web con selenium y BeautifulSoup de las páginas de eDreams y Kayak.

In [ ]:
# Driver de Chrome
chrome_driver = "chromedriver.exe"

### eDreams
**`eDreams`** nos permite una busqueda abierta donde a partir de un origen y unas fechas de ida/vuelta, te ofrece 15 destinos sobre los que mediante scraping podemos obtener gran cantidad de vuelos y combinaciones.

*__Link__: https://www.edreams.es*

In [ ]:
#Funciones

#Funcion principal del scrapeo a edreams. Gestiona todas las acciones necesarias para scrapear (selenium, diferentes urls, soap, preparar y devolver datos)
def scrapping_edreams( origen, inicio, fin):
    print(f"Procesando {origen} - {inicio} to {fin}")
    url = f"https://www.edreams.es"
    
    #Llamada a la funcion que se encarga de hacer las acciones necesarias mediante selenium para lanzar la busquesda inicial por origen + fechas
    soup = obtener_posibles_destinos(url=url, origen=origen, inicio=inicio, fin=fin)

    #Generar las URLs de destinos+fechas basado en el contenido de la etiqueta article y la plantilla de url
    #"{url}/travel/#results/type=R;dep={inicio};from={origen};to={destino};ret={fin};collectionmethod=false"
    destinos = soup.find_all("article", class_ = "od-inspirational-grid-col")
    #obtenemos los codigos IATA sobre los que luego buscaremos al intruducirlos como parametros en la URL
    lista_destinos = [destino.find("figure")["data-iata"] for destino in destinos]
    
    urls_destinos = {}
    for destino in lista_destinos:
        #url_template = f"{url}/travel/#results/type=R;dep={inicio};from={origen};to={???};ret={fin};collectionmethod=false"
        urls_destinos[destino] = f"{url}/travel/#results/type=R;dep={inicio};from={origen};to={destino};ret={fin};collectionmethod=false"

    #scrap data
    datos_scrapeados = []
    for destino,destino_url in tqdm.tqdm(urls_destinos.items(), total=len(urls_destinos)):
        #datos fijos que sabemos por la propia busqueda: url, origen, destino, inicio, fin
        fixed_data = [destino_url, origen, destino, inicio, fin]

        #datos obtenidos del scrapeo de la url con los vuelos a ese destino
        data_destino = datos_destino(url=destino_url)

        #list comprehension para nutrir cada elemento con los valores fijos
        full_data_destino = [fixed_data + rd for rd in data_destino]
    
        datos_scrapeados.extend(full_data_destino)

    return datos_scrapeados

#Funcion que realiza con selenium todas las acciones dinamicas para poder obtener todos los posibles destinos en base a un origen y unas fechas
def obtener_posibles_destinos(url, origen, inicio, fin):

    browser = webdriver.Chrome()
    browser.get(url)
    sleep(1)
    browser.maximize_window()
    sleep(1)

    try:
    
        #aceptar cookies
        browser.find_element(By.ID, "didomi-notice-agree-button").click()
        sleep(1)

        #Escribir en el inputo de origen el valor recibido
        browser.find_element(By.XPATH, '//input[@test-id="input-airport"]').send_keys(origen)
        sleep(1)

        #Click en el origen que se muestra como resultado del paso anterior
        browser.find_element(By.XPATH, '//div[@test-id="airport-departure"]').find_element(By.XPATH, f".//ul/li/div/span[contains(text(), \"{origen}\")]").click()
        sleep(1)

        #Click en la primera opcion de destino que se muestra al hacer el paso anterior, que es cualquier destino
        browser.find_element(By.XPATH, '//div[@test-id="airport-destination"]').find_element(By.XPATH, './/div/div/ul/li/div[contains(@class, "odf-dropdown-col") and contains(@class, "lg") and contains(@class, "odf-text-nowrap")]').click()
        sleep(1)

        #Logica para abrir el calendario y elegir las fechas que hemos recibido, fecha de inicio
        div_calendario_salida = browser.find_element(By.XPATH, '//div[@data-testid="departure-date-picker"]')
        procesar_calendario(fecha=inicio, element=div_calendario_salida)
        sleep(1)

        #Logica para abrir el calendario y elegir las fechas que hemos recibido, fecha de fin
        div_calendario_vuelta = browser.find_element(By.XPATH, '//div[@data-testid="return-date-picker"]')
        procesar_calendario(fecha=fin, element=div_calendario_vuelta)
        sleep(1)

        #Click en el boton Continuar para confirmar las fechas (y todo lo previo)
        div_calendario_vuelta.find_element(By.XPATH, ".//div/button[contains(text(), \"Continuar\")]").click()

        #Lanzar la busqueda de destinos
        boton_buscar = browser.find_element(By.XPATH, '//button[@test-id="search-flights-btn"]')
        boton_buscar.click()
        sleep(10)

        #Una vez ha cargado los resultados, lo montamos en BS y lo devolvemos
        soup = BeautifulSoup(browser.page_source, "html.parser")
        browser.quit()

        if soup is None:
            raise Exception

        return soup
    
    except Exception as exception:
        print(f"Excepcion buscando destinos... {exception}")
        return None

#Funcion para parsear la fecha recibida a un formato especial que hay en la pagina
def custom_ano_mes_format( fecha = datetime.now()):

    locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')
    #Obtener el nombre del mes en formato completo (por ejemplo, "noviembre")
    parsed_month = fecha.strftime('%B').capitalize()

    #Obtener los dos últimos dígitos del año (por ejemplo, "23" en lugar de "2023")
    parsed_year = fecha.strftime('%y')

    #Formatear la fecha en el formato deseado "Mes 'YY"
    formatted_fecha = f"{parsed_month} '{parsed_year}"
    
    return formatted_fecha

#Funcion para mostrar el mes deseado, via selenium
def mostrar_mes(ano_mes_str, html_element):
    while True:
        #Obtener los calendarios de los meses que actualmente vemos en la pagina
        meses_visibles = [e.text for e in html_element.find_elements(By.CSS_SELECTOR, "div.odf-calendar-title")]

        #Comprobamos si tenemos visible el mes que queremos
        if ano_mes_str in meses_visibles:
            #Si el mes que buscamos esta en pantalla, salimos
            break
        else:
            #Si no esta visible el mes, hacemos click en el boton de siguiente mes y volvemos al inicio del while
            html_element.find_element(By.XPATH, './/div/div/div/button/span[contains(@class, "odf-icon-arrow-right")]').click()

    return

#Funcion para procesar las acciones necesarias con selenium para mostrar el calendario
def procesar_calendario(fecha, element):
    #A partir de la fecha recibida, transformamos a formato "Mes 'YY" que es lo que la web muestra y por lo tanto hay que buscar
    fecha_datetime = datetime.strptime(fecha, "%Y-%m-%d")
    fecha_ano_mes = custom_ano_mes_format(fecha_datetime)

    #La pagina nos muestra 2 meses, el actual y el siguiente, pero debemos buscar realmente el que corresponda a la fecha recibida
    mostrar_mes(ano_mes_str=fecha_ano_mes, html_element=element)

    #Una vez tenemos visible el mes que queremos, buscamos el mes
    calendario_fecha = element.find_element(By.XPATH, f".//div[contains(@class, \"odf-calendar-title\") and contains(text(), \"{fecha_ano_mes}\")]/following-sibling::div")

    #Buscamos el dia dentro de ese mes, y le hacemos click
    dia_fecha = calendario_fecha.find_element(By.XPATH, f".//div[contains(@class, \"odf-calendar-day\") and contains(text(), \"{fecha_datetime.day}\")]")

    #Le hacemos click
    dia_fecha.click()

#Funcion para detectar y quitar una alerta/boton molesto
def check_boton_molesto(browser):
    #check stupid alert
    try:
        stupid_alert = browser.find_element(By.ID, "sessionAboutToExpireAlert")

        if stupid_alert:
            stupid_button = stupid_alert.find_element(By.CCS_SELECTOR, "button")
            if stupid_button:
                stupid_button.click()
                sleep(1)
                return True
    except Exception:
        # print("Error detectando el boton estupido...")
        return False

    return False

#Funcion para scrapear con selenuim y BS el detalle de los vuelos segun la url recibida que contiene ya el conjunto de datos de origen, destino, inicio y fin
def datos_destino(url):
    print(f"Processing {url}")
    lista_datos_destino = []
    
    browser = webdriver.Chrome()
    browser.get(url)
    sleep(1)
    browser.maximize_window()
    sleep(15)

    #Aceptar cookies
    browser.find_element(By.ID, "didomi-notice-agree-button").click()
    sleep(1)

    #Bucle para hacer scroll y clieck en mostrar mas resultados, hasta que no se pueda hacer mas scroll
    counter = 0
    scroll = 10000
    while True:
        #Scroll
        browser.execute_script(f"window.scrollBy(0, {scroll});")
        sleep(6)
        #Checkeamos si existe un boton molesto, y lo quitamos
        check_boton_molesto(browser=browser)

        #Buscamos los botones
        botones = browser.find_element(By.ID, "results_list_container").find_elements(By.XPATH, ".//button")

        #Si hay botones y el ultimo boton contiene "Mostrar ", le damos click
        if len(botones) and botones[-1] and "Mostrar " in botones[-1].text:
            # print("Click en " + botones[-1].text)
            try:
                botones[-1].click()
            except Exception:
                #En caso de error, checkeamos si existe el boton molesto, y lo quitamos
                if check_boton_molesto(browser=browser):
                    print(f"Boton estupido detectado y clickado :)")
                    continue
                else:
                    #Si hay error desconocido, simplemente dejamos de hacer scroll y pasamos a extraer datos
                    break

            counter += 1
            #Cada bucle aumentamos el scroll
            scroll += 500

        elif not check_boton_molesto(browser=browser):
            #Si no tengo botones ni boton molesto, dejo de hacer scroll
            break

    sleep(4)
    print(f"Scroll hecho {counter} veces")


    #Montamos el BS con el contenido
    soup = BeautifulSoup(browser.page_source, "html.parser")
    browser.quit()

    #Contenedor con todos los divs de vuelos
    results_container = soup.find(id = "results_list_container")
    elements = results_container.find_all(attrs={"data-testid" : "itinerary"})

    for element in elements:
        duraciones = []
        escalas = []
        datos_horas = []
        equipajes = []

        try:
            #Aeropuertos, es una lista con los 2 de la ida y los 2 de la vuelta
            aeropuertos = element.find_all('div', {'type': 'small'})
            aeropuertos_data = [aeropuertos[0].text.strip(),aeropuertos[2].text.strip(),aeropuertos[4].text.strip(),aeropuertos[6].text.strip()]
            
            #Aerolineas, las dejamos en una lista de valores unicos
            aerolineas_elem = element.find_all('img')
            aerolineas = list({a.attrs["alt"] for a in aerolineas_elem if 'alt' in a.attrs})
            
            #Precios. El bueno es el unit_price, pero tambien hay otros precios con descuento que pueden servir
            price_spans = element.find_all('span', class_ = "money-integer")
            prices = [p.text for p in price_spans]
            unit_price = element.select("a > span > span.money-integer")[0].text

            #Variable para tener todos los divs del vuelo
            divs_vuelo = element.find_all('div')

            #Bucle para iterar por todos los divs y sacar los diferentes datos
            for div_item in divs_vuelo:
                #Logica para sacar las horas de despegue y llegada
                if len(div_item.attrs) == 1 and "class" in div_item.attrs:
                    #Al ser clases css dinamicas, hay que recorrerlas y buscar la que acabe en BaseText-Body que es estatico. 
                    for attr_class in div_item.attrs["class"]:
                        #Regex para buscar en el contenido del div que tenga un formato de XX:XX
                        if attr_class.endswith("BaseText-Body") and re.match(r'^\d{2}:\d{2}$', div_item.text.strip()):
                            datos_horas.append(div_item.text)
                #Logica para sacar datos de escalas, a partir de un atributo orientation que tiene ese elemento
                elif len(div_item.attrs) > 1 and "orientation" in div_item.attrs:
                    #Es imposible detectar directamente el elemento, por eso buscamos el anterior que si podemos encontrar, y vamos al siguiente sibling
                    next_sibling = div_item.findNextSibling()
                    if next_sibling:
                        #El span contiene las duraciones y las escalas
                        span_items = next_sibling.find_all("span")
                        duraciones.append(span_items[0].text)
                        escalas.append( span_items[1].text if len(span_items) > 1 else "0" )

            #Bucle para iterar por todos los elementos path y sacar datos
            child_paths = element.find_all('path')
            for path_item in child_paths:
                #clip-rule es algo fijo que siempre podremos encontrar
                if len(path_item.attrs) > 1 and "clip-rule" in path_item.attrs:
                    #ojo sensibles: buscamos tres veces el parent
                    tri_parent = path_item.parent.parent.parent
                    if tri_parent:
                        #buscamos el siguiente elemento
                        equipaje_div = tri_parent.findNextSibling() 
                        if equipaje_div:
                            #agregamos la info de maletas
                            equipajes.append(equipaje_div.text)

        except Exception as exception:
            print(f"Ignorando vuelo por problemas al scrapear...{exception}")
            continue

        lista_datos_destino.append( [aeropuertos_data,aerolineas,datos_horas,duraciones,escalas,equipajes,unit_price,prices] )

    return lista_datos_destino


AIRTABLE_BASE_URL = "https://api.airtable.com/v0"
BASE_ID = "appeoVItHkVNqxCPe" # Base: Tabla API
TABLE_ID  = "tblc9PqQwMxECrPf0" # Tabla: Data Base

#Funcion para subir a airtables el df
def subir_datos_airtable(df):
    API_KEY = os.getenv("AIRTABLE_API_KEY_SHARED") #API KEY de AirTable cargada desde el ordenador mediante un fichero .env

    # Headers - Credenciales para hacer solicitudes mediante API en AirTable
    headers = {"Authorization" : f"Bearer {API_KEY}",
            "Content-Type"  : "application/json"}

    # Formateos para evitar errores
    df1 = df.replace({"" :  None})
    df1 = df1.replace({np.nan :  None})

    datos_df = []

    for idx, row in df1.iterrows():
        data = {'fields': row.to_dict()}
        datos_df.append(data)

    # endpoint
    endpoint = f"{AIRTABLE_BASE_URL}/{BASE_ID}/{TABLE_ID}"

    counter = 0
    while counter < len(datos_df):
        datos_subir = datos_df[counter:counter+10]
        datos_subir = {'records': datos_subir, 'typecast': True }
        
        response = requests.post(url = endpoint, json = datos_subir, headers = headers)

        (f"response: {response.status_code}")
        # print(f"endpoint: {response.url}")
        # print("-"*120)
        counter += 10
        sleep(1)

    print(f"Subidos {len(datos_df)} registros a airtables")
    
#Funcion para crear el df con los datos scrapeados de edreams
def crear_df(data):
    columnas_df = ['url', 'origen', 'destino', 'fecha_inicio', 'fecha_fin', 'pasajeros', 'inicio_ida', 'fin_ida', 'inicio_vuelta', 'fin_vuelta', 'escala_ida', 'escala_vuelta', 'duracion_ida', 'duracion_vuelta', 'aerolineas', 'equipaje_mano', 'equipaje_bodega', 'precio', 'clase']
    df = pd.DataFrame(data, columns = columnas_df)

    #Ajusta las columnas de escalas para que sea solo el numero. De origen viene directo o X escalas
    df['escala_ida'] = df.apply(lambda row: int(row["escala_ida"][:1] if row["escala_ida"] != "directo" else 0), axis=1)
    df['escala_vuelta'] = df.apply(lambda row: int(row["escala_vuelta"][:1] if row["escala_vuelta"] != "directo" else 0), axis=1)

    #Ajusta las columnas de duracion para que el formato sea 1h 35m en vez de 1 h 35 min
    df['duracion_ida'] = df.apply(lambda row: row['duracion_ida'].replace(" h","h").replace(" min","m"), axis=1)
    df['duracion_vuelta'] = df.apply(lambda row: row['duracion_vuelta'].replace(" h","h").replace(" min","m"), axis=1)

    #guardar en pkl el df con un timestamp
    now = datetime.now()
    now_timestamp = now.strftime("%Y%m%d%H%M%S")
    file_name = f"edreams_data_{now_timestamp}"
    df.to_pickle(f"{file_name}.pkl")
    print(f"Creado {file_name}.pkl")## Ejecución del proceso

    return df

##### Ejecución del proceso de scrapeo:

In [ ]:
#Lista/Diccionario con el conjunto de fechas a Scrapear
fechas = [
    {
        "inicio" : "2023-12-06",
        "fin" : "2023-12-10"
    },
    {
        "inicio" : "2023-12-22",
        "fin" : "2023-12-26"
    },
    {
        "inicio" : "2023-12-29",
        "fin" : "2024-01-02"
    },
    {
        "inicio" : "2024-01-05",
        "fin" : "2024-01-09"
    }
]

#Lista de origenes a scrapear
origenes = ["MAD","PMI","LCG"]

#Bucle para recorrer cada uno de los origenes
for origen in origenes:
    #Bucle para recorrer cada una de las fechas
    for date in fechas:
        data = []
        #Llamada a la funcion que, en base al origen + fechas, lanza todo el scrapeo necesario y nos devuelve una lista de valores
        lista_datos_obtenidos = scrapping_edreams( origen = origen, inicio = date['inicio'], fin = date['fin'])

        #Bucle para recopilar los datos obtenidos del scrapeo, setear algunos datos fijos, parsear y tener el conjunto de datos finales para generar el df
        for datos_obtenidos in lista_datos_obtenidos:
            url = datos_obtenidos[0]
            origen = datos_obtenidos[1]
            destino = datos_obtenidos[2]
            fecha_inicio = datos_obtenidos[3]
            fecha_fin = datos_obtenidos[4]
            pasajeros = 1 #valor fijo
            inicio_ida = datos_obtenidos[7][0]
            fin_ida = datos_obtenidos[7][1]
            inicio_vuelta = datos_obtenidos[7][2]
            fin_vuelta = datos_obtenidos[7][3]
            escala_ida = datos_obtenidos[9][0]
            escala_vuelta = datos_obtenidos[9][1]
            duracion_ida = datos_obtenidos[8][0]
            duracion_vuelta = datos_obtenidos[8][1]
            aerolineas = datos_obtenidos[6]
            equipaje_mano_value = datos_obtenidos[10][0] if len(datos_obtenidos[10])>1 else "" #En edreams solo podemos detectar equipaje de mano, buscamos ese valor y sino 0
            equipaje_mano = 1 if equipaje_mano_value == "Equipaje de mano" else 0
            equipaje_bodega = 0 #no existe este valor en edreams
            precio = datos_obtenidos[11]
            clase = None #no existe este valor en edreams

            data.append([url,origen, destino, fecha_inicio, fecha_fin, pasajeros, inicio_ida, fin_ida, inicio_vuelta, fin_vuelta, escala_ida, escala_vuelta, duracion_ida, duracion_vuelta, aerolineas, equipaje_mano, equipaje_bodega, precio, clase])
        
        if len(data)>0:
            #Creamos el df
            data_df = crear_df(data=data)
            #Subimos el df a airtables
            subir_datos_airtable(df=data_df)
    print("===================== FIN DEL PROCESO =====================")

### Kayak

**`kayak`** nos permite una busqueda abierta donde a partir de un origen y unas fechas de ida/vuelta, te ofrece 15 destinos sobre los que mediante scraping podemos obtener gran cantidad de vuelos y combinaciones.

*__Link__: https://www.kayak.es*

In [2]:
"""Diccionario key:value de Ciudades con su código IATA

Los códigos IATA consisten en un código de tres letras que suele tener relación con la ciudad o región 
a la que sirve el aeropuerto, o bien el nombre del aeródromo existente antes de la creación del aeropuerto,
pero que no tiene por qué contener información geográfica del mismo."""

aeropuertos_espana = {
    "Madrid": "MAD",
    "Barcelona": "BCN",
    "Valencia": "VLC",
    "Sevilla": "SVQ",
    "Malaga": "AGP",
    "Bilbao": "BIO",
    "Mallorca": "PMI",
    "GranCanaria": "LPA",
    "TenerifeNorte": "TFN",
    "TenerifeSur": "TFS",
    "Alicante": "ALC"
}

#Diccionario con claves de ciudades, valor códigos IATA de los aeropuertos de respectivas ciudades
with open("iata.pkl", "br") as file:
    ciudades_iata = pickle.load(file)

In [8]:
origen = aeropuertos_espana['Madrid']

destinos = []

#Se visita la pag web donde estarán todos los destino posibles de ese momento

url_visitar = f'https://www.kayak.es/explore/{origen}-anywhere'

browser = webdriver.Chrome()

browser.get(url=url_visitar)

browser.maximize_window()

sleep(1)

browser.find_element(By.CLASS_NAME, 'RxNS-button-container').click() #Esta línea de código acepta términos y condiciones de la web/cookies.

sleep(1)

In [9]:
def cargar_destinos():#Esta función carga las páginas o demás resultados que arroja la web al buscar una ruta de un origen a un destino cualquiera, suele arrojar hasta 100-120 destinos

    #Como acceder al botón mediante algún patrón en el html del sitio es bastante complejo se opta por hacerlo mediante XPATH, aún así en el diversos tests se ha encontrado que el botón tiene otras posibles ubicaciones
    
    while True:
        try:
            boton_cargar_destinos = browser.find_element(By.XPATH, '/html/body/div[1]/div[1]/main/div[2]/div[2]/div[2]/div/div[2]/div/div/div/div[6]/button')
            boton_cargar_destinos.click()
            sleep(2.5)
            print('Try - 1')

        except:
            try:
                boton_cargar_destinos = browser.find_element(By.XPATH, '/html/body/div[2]/div[1]/main/div[2]/div[2]/div[2]/div/div[2]/div/div/div/div[6]/button')
                boton_cargar_destinos.click()
                sleep(2.5)
                print('Try - 2')

            except:
                print('No hay más destinos que cargar')
                break

In [10]:
cargar_destinos()

Try - 2
Try - 2
Try - 2
Try - 2
Try - 2
Try - 2
Try - 2
No hay más destinos que cargar


In [ ]:
#Una vez conseguidas todas las ciudades que tiene la web para mostrarnos, se hace un BeautifulSoup para extraer los nombres de las ciudades

soup = BeautifulSoup(browser.page_source, 'html.parser') #Se crea el soup para tener el html de la web y poder explorar en ella

ciudades = soup.find_all('div', class_ = 'City__Name') #Se consiguen todos los elementos que tienen el nombre de la ciudad

ciudades = [ciudad.text for ciudad in ciudades] #Se guarda en una lista el string que contiene los elementos encontrados en la linea anterior

print(f'Se han encontrado {len(ciudades)} posibles destinos desde {origen}')

In [ ]:
def buscar_codigo_iata(ciudad): #Con esta función se busca el nombre de la ciudad que falta en el diccionario ciudades_iata, consultando la web oficial de IATA (tienen una API que lo hace pero sigue en desarrollo)
    
    with open("iata.pkl", "br") as file:
        ciudades_iata = pickle.load(file)
    
    try:
        url_query = f'https://www.iata.org/en/publications/directories/code-search/?airport.search={ciudad}'

        browser.get(url=url_query)

        soup = BeautifulSoup(browser.page_source, 'html.parser')

        codigo_iata = soup.find('table', class_ = 'datatable').find('td', attrs={'data-heading': '3-letter location code'}).text

        ciudades_iata[ciudad] = codigo_iata
        
        sleep(1)
        
    finally:
        
        with open("iata.pkl", "bw") as file:
            pickle.dump(ciudades_iata, file)
        
        return ciudades_iata[ciudad]

In [ ]:
with open("iata.pkl", "br") as file:
    ciudades_iata = pickle.load(file)
    
#Esta list comprehension ayuda a saber qué resultados de la búsqueda anterior no están en el diccionario cargado del archivo pickle donde están los códigos IATA

ciudades_faltantes = [ciudad for ciudad in ciudades if ciudad not in ciudades_iata]

for ciudad in ciudades_faltantes:

    try:
        buscar_codigo_iata(ciudad) #Con esta función se busca el nombre de la ciudad que falta en el diccionario consultando la web oficial de IATA
    
    except:
        print(f'No se ha encontrado {ciudad} en la búsqueda y se ha eliminado de la lista de ciudades destino') #Si no encuentra el código IATA de la fuente oficial se descarta el posible destino
        ciudades.remove(ciudad)

In [ ]:
def cargar_resultados(): #Esta función se emplea para cargar en la página de vuelos de origen-destino en el rango de fechas deseado.
    
    counter = 0
    
    #Cada página tiene un máximo de 15 resultados, al cargar todas las páginas hasta 10 se podrán sacar hasta 150 vuelos por ruta (puede haber menos)
    while counter<10: #Si se desea sacar todos los vuelos de una ruta este código se puede cambiar a un while True
        
        try:
            boton_cargar_resultados = browser.find_element(By.XPATH, '/html/body/div[2]/div[1]/main/div/div[2]/div[2]/div[1]/div[2]/div[1]/div[3]/div[1]/div/div/div')
            boton_cargar_resultados.click()
            sleep(3)
            counter += 1
            #print(f'Cargando más resultados {counter}')
            
        except:
            
            print(f'Se cargaron {counter} páginas adicionales')
            #print('No hay más destinos que cargar')
            
            break
        
    print(f'Carga completa')

In [ ]:
def extraer_vuelos(destino, fecha_inicio, fecha_fin, pasajeros = 1): #El parámetro pasajeros es opcional y definido a 1, si se quisiera cambiar el valor mediante un input o usando otra variable con un valor asignado
    
    counter = 0
    for idx, vuelo in enumerate(vuelos):
        try:
            if counter % 50 == 0:
                print(f'Estraidos datos de {counter} vuelos')
            
            #Datos del viaje
            
            #Horarios
            horario = vuelo.find('div', class_ = 'nrc6-inner').find_all('div', class_ = 'VY2U')
            horario = [viaje.find('div').text for viaje in horario]
            inicio_ida, fin_ida = horario[0].split(' – ')
            inicio_vuelta, fin_vuelta = horario[1].split(' – ')

            #Escalas
            escalas = vuelo.find('div', class_ = 'nrc6-inner').find_all('div', class_ = 'JWEO')
            escala_ida, escala_vuelta = [int(parada.text[0]) if parada.text != 'directo' else 0 for parada in escalas]

            #Duración vuelos
            duraciones = vuelo.find('div', class_ = 'nrc6-inner').find_all('div', class_ = 'JWEO')
            duracion_ida, duracion_vuelta = [duracion.find_next('div').find_next('div').find_next('div').text for duracion in duraciones]
                
            #Aerolinea
            aerolineas = vuelo.find_all('img')
            aerolineas = list({str(aerolinea).split('"')[1] for aerolinea in aerolineas})   
                
            #Equipaje y Precio

            equipaje_mano = vuelo.find('div', class_ = 'nrc6-price-section').text[0]
            equipaje_bodega = vuelo.find('div', class_ = 'nrc6-price-section').text[1]
            precio = float(vuelo.find('div', class_ = 'nrc6-price-section').text[2:].replace('\xa0','').split('€')[0])
            clase = vuelo.find('div', class_ = 'nrc6-price-section').text[2:].replace('\xa0','').split('€')[1].replace('Seleccionar','')

            data.append([url,origen, destino, fecha_inicio, fecha_fin, pasajeros, inicio_ida, fin_ida, inicio_vuelta, fin_vuelta, escala_ida, escala_vuelta, duracion_ida, duracion_vuelta, aerolineas, equipaje_mano, equipaje_bodega, precio, clase])

            counter += 1
            
        except Exception as e:
            print(f"Problema vuelo {idx}. {e}")
            
    print(f'Se han extraido de manera exitosa datos de {counter} vuelos de {len(vuelos)} | Accuracy {(counter/len(vuelos))*100}%')
        

In [ ]:
#De acuerdo con los datos que se han podido sacar del scrapeo se definen las columnas que se van a utilizar en el data frame
columnas_df = ['url', 'origen', 'destino', 'fecha_inicio', 'fecha_fin', 'pasajeros', 'inicio_ida', 'fin_ida', 'inicio_vuelta', 'fin_vuelta', 'escala_ida', 'escala_vuelta', 'duracion_ida', 'duracion_vuelta', 'aerolineas', 'equipaje_mano', 'equipaje_bodega', 'precio', 'clase']

#Se crea una lista vacía donde se va a guardar toda la información del scrapeo
data = []

#Esta variable es opcional ya que en la función anterior tiene asignada un 1 por defecto, se puede reemplazar con un input para pedir al usuario que ingrese el numero de pasajeros, pero en este caso se simulará con 1 pasajero
pasajeros = 1

#Aquí está la lista de fechas con un rango en una tupla (fecha_inicio, fecha_fin) de donde se van a extraer los datos mediante un bucle for
fechas = [('2023-12-22','2023-12-26'), ('2023-12-29','2024-01-02')]

In [ ]:
with open("iata.pkl", "br") as file: #Se abre el diccionario con los códigos IATA para que se pueda iterar sobre la URL de Kayak
    ciudades_iata = pickle.load(file)
    
for idx, fecha in enumerate(fechas): #Se itera sobre la lista de rango de fechas que se quieren sacar los datos (cada elemento de esta lista son tuplas, que a su vez está conformada por un par de fechas en formato string)
    
    fecha_inicio = fecha[0]
    fecha_fin = fecha[1]
    fecha
    
    for idx, ciudad in tqdm.tqdm(enumerate(ciudades)): #Se itera sobre la lista de posibles destinos que se encontraron en la primera búsqueda de posibles destinos, y quitando los destinos no encontrados en la consulta a la web de IATA

        destino = ciudades_iata[ciudad]

        url = f'https://www.kayak.es/flights/{origen}-{destino}/{fecha_inicio}/{fecha_fin}/{pasajeros}adults?sort=price_a'

        try:

            browser.get(url)
            sleep(30)
            cargar_resultados()

            soup = BeautifulSoup(browser.page_source, 'html.parser')
            vuelos = soup.find_all('div', class_ = 'nrc6')

            extraer_vuelos(destino, fecha_inicio, fecha_fin, pasajeros) #Esta función tiene como parámetro opcional "pasajeros", si le pasamos esa variable antes con un input podemos sacar datos haciendo una búsqueda para 2 pasajeros

            if idx % 2 == 0 or idx == (len(ciudades)): #Este código sirve para guardar el progreso de los scrappeado a medida que avanza el buble for
                df = pd.DataFrame(data, columns = columnas_df)
                df.to_pickle('kayak_webscraping_temp.pkl')
                print(len(data))

        except:

            print(f'Ha fallado la extracción de datos de la ruta {origen}-{destino}/{fecha_inicio}/{fecha_fin}')
        
df = pd.DataFrame(data, columns = columnas_df)
df.to_pickle('kayak_webscraping.pkl')
df.to_csv('kayak_webscraping.csv', index = False)

### Carga de datos de Kayak a AirTable

In [ ]:
print(load_dotenv())

API_KEY = os.getenv("AIRTABLE_API_KEY") #API KEY de AirTable cargada desde el ordenador mediante un fichero .env

BASE_ID = "appeoVItHkVNqxCPe" # Base: Tabla API

TABLE_ID = "tblc9PqQwMxECrPf0" # Tabla: Data Base

airtable_base_url = "https://api.airtable.com/v0"

# Headers - Credenciales para hacer solicitudes mediante API en AirTable
headers = {"Authorization" : f"Bearer {API_KEY}",
           "Content-Type"  : "application/json"}

API_KEY

In [ ]:
df = pd.read_pickle('kayak_webscraping.pkl')

In [ ]:
df.isna().sum()

In [ ]:
df = df.replace({"" :  None})
df = df.replace({np.nan :  None})
df.info()

In [ ]:
datos_df = []

for idx, row in df.iterrows():
    data = {'fields': row.to_dict()}
    datos_df.append(data)

In [ ]:
# Crear records en la tabla AirTable

# Endpoint
endpoint = f"{airtable_base_url}/{BASE_ID}/{TABLE_ID}"

counter = 0

while counter < len(datos_df):
    
    if counter % 50 == 0:
        print(f'Subiendo de {counter} de {len(datos_df)}')
        
    datos_subir = datos_df[counter:counter+10]
    datos_subir = {'records': datos_subir, 'typecast': True }
    
    response = requests.post(url = endpoint, json = datos_subir, headers = headers)

    print(f"response: {response.status_code}")
    print(f"endpoint: {response.url}")
    print("-"*120)
    counter += 10
    sleep(1)

## Lectura de los datos

In [ ]:
print(load_dotenv())

API_KEY = os.getenv("AIRTABLE_API_KEY") #API KEY de AirTable cargada desde el ordenador mediante un fichero .env

BASE_ID = "appeoVItHkVNqxCPe" # Base: Tabla API

TABLE_ID = "tblc9PqQwMxECrPf0" # Tabla: Data Base

airtable_base_url = "https://api.airtable.com/v0"

# Headers - Credenciales para hacer solicitudes mediante API en AirTable
headers = {"Authorization" : f"Bearer {API_KEY}",
           "Content-Type"  : "application/json"}

API_KEY

In [ ]:
#Descargar los datos de AirTable

endpoint = f"{airtable_base_url}/{BASE_ID}/{TABLE_ID}"

params = {}

datos = []

while True:
    
    response = requests.get(url = endpoint, headers = headers, params = params)
    
    data = response.json()

    datos.extend(data['records'])
    
    offset = data.get("offset")

    if offset is None:
        break
    
    params["offset"] = offset
    
    # sleep(1)
    print(f'Descargados {len(datos)} datos')

In [ ]:
df = pd.DataFrame([x['fields'] for x in datos])

In [ ]:
df.head(3)

## Análisis de datos


### 1. Limpieza de los datos:

In [ ]:
#En caso de querer ejecutar las visualizaciones sin descargar los datos que se encuentran en AirTable, ejecutar esta línea de código (el archivo .pkl estará adjunto en el mail)
df = pd.read_pickle('airtable.pkl')

In [ ]:
#En primer lugar añadiré una nueva columna para definir si los datos son de kayak o de edreams

def asignar_pagina_web(url):
    if "kayak" in url.lower():
        return "kayak"
    elif 'edreams' in url.lower():
        return "edreams"
    else:
        return "otro"

# Crear una nueva columna 'pagina_web' usando la función asignar_pagina_web
df["pagina_web"] = df["url"].apply(asignar_pagina_web)

In [ ]:
#Luego creamos dos columnas nuevas en las que a partir del iata.pkl transformamos los códigos iata del origen y destino al nombre de la ciudad.
def convertir_iata(codigo):
    
    with open("iata.pkl", "br") as file:
        ciudades_iata = pickle.load(file)
        
    for nombre_ciudad, codigo_iata in ciudades_iata.items():
        if codigo == codigo_iata:
            
            return nombre_ciudad

df[["ciudad_origen","ciudad_destino"]] = df[["origen","destino"]].apply(lambda codigo: codigo.apply(convertir_iata))

In [ ]:
df["ciudad_destino"].value_counts() #comprobamos que el código funciona

In [ ]:
datetime_columns = [column for column in df.columns if "inicio" in column or "fin" in column]
df[datetime_columns]

In [ ]:
def convertir_duracion_a_timedelta(duracion):
    partes = duracion.split(' ')
    horas = 0
    minutos = 0

    if "h" in partes[0]:
        horas = int(partes[0].split('h')[0])

    if "m" in partes[-1]:
        minutos = int(partes[-1].split('m')[0])

    return timedelta(hours= horas, minutes= minutos)


#Aplicamos la función a las columnas 'duracion_ida' y 'duracion_vuelta' del DataFrame df
df["duracion_ida"] = df["duracion_ida"].apply(convertir_duracion_a_timedelta)
df["duracion_vuelta"] = df["duracion_vuelta"].apply(convertir_duracion_a_timedelta)

In [ ]:
datetime_columns = [column for column in df.columns if "inicio" in column or "fin" in column]

# Limpieza de celdas que contienen "+"
df[['fin_ida','fin_vuelta']] = df[['fin_ida','fin_vuelta']].apply(lambda hora: hora.str.split('+').str[0])

for column in datetime_columns:
    
    if "fecha" not in column:
        df[column] = pd.to_datetime(df[column]).apply(lambda x : x.time())
        
    else:
        df[column] = pd.to_datetime(df[column])

In [ ]:
integer_columns = [column for column in df.columns if "pasajeros" in column or "escala" in column or "equipaje" in column]

for column in integer_columns:
    df[column] = df[column].astype("int64")

In [ ]:
df["num_aerolineas"] = df["aerolineas"].apply(lambda x: len(x))

In [ ]:
df = df.explode("aerolineas")

In [ ]:
df["aerolineas"].value_counts()

In [ ]:
#Finalmente me quedaré sólo con las fechas de navidad para comparar los resultados de ambas páginas web
df = df[~df["fecha_inicio"].isin(["2023-12-06", "2024-01-05"])]
#Además, sólo analizaremos los resultados que salgan de Madrid
df = df[df['origen'] == 'MAD']

In [ ]:
df.dtypes # Así terminamos con un df en el cuál podremos extraer información de los vuelos que nos ofrece cada
          # una de las páginas web en función de la fecha de salida, de vuelta, la duración del vuelo, los horarios de salido y vuelta
          # así como las condiciones del vuelo.

### 2. Preguntas sobre los datos:

In [ ]:
df.head(3)

Observamos la variedad de precios que se obtienen por destino saliendo desde Madrid en las fechas de Navidad y Año Nuevo.
Para acotar estos resultados nos haremos ciertas preguntas específicas sobre los datos para poder el elegir el vuelo que más nos convenga para estas vacaciones.

In [ ]:
# Distribución de precios por ciudad de origen y destino
fig = px.box(df, x = "ciudad_origen", y = "precio", color="ciudad_destino",
             labels = {"precio": "Precio", "ciudad_origen": "Ciudad de Origen", "ciudad_destino": "Ciudad de Destino"},
             title = "Distribución de precios de vuelos por ciudad de origen y destino")
fig.show()


Al analizar los datos hemos observado que entre las aerolíneas también estaban compañías no aéreas, por lo que filtramos el df para quedarnos sólo con las vuelos. Consideramos interesante evaluar la proporción de esos trayectos no aéreos que nos dan las páginas web.

In [ ]:
#Primero contabilizamos el número de vuelos y de los otros transportes
otros_transportes = ['SNCF','Segesta Autolinee', 'iryo','Coastal', 'Renfe','Flibco','BlaBlaBus', 'ALSA','FlixBus',
                     'Trenitalia','Autobús','Sagalés','Gipsyy', 'GoOpti', 'Socibus', 'AccessRail', 'Union Ivkoni']

num_vuelos = len(df) - df.explode('aerolineas')['aerolineas'].isin(otros_transportes).sum()
num_otros_transportes = df.explode('aerolineas')['aerolineas'].isin(otros_transportes).sum()


#Creamos una lista y etiquetamos las categorías
valores = [num_vuelos, num_otros_transportes]
categorias = ['vuelos', 'otros transportes']

In [ ]:
#Realizamos una visualización del resultado
plt.figure(figsize=(8, 6))
plt.pie(valores, labels=categorias, autopct='%1.1f%%', startangle=140)
plt.title('Proporción de vuelos y otros transportes')
plt.show()


Del total de vuelos que nos ofrecen las dos páginas webs, un 2.2% se corresponden a viajes en trenes, buses y afines.

In [ ]:
#Tras comprobar que no sólo tenemos viajes operados por vuelos, sino también por trenes, autobuses, etc. Deseamos eliminar esas
#compañías para sólo tener los vuelos
def eliminar_aerolineas(df, lista_aerolineas):
    df = df[~df['aerolineas'].isin(lista_aerolineas)]
    return df

In [ ]:
# Ejemplo de uso:
# Llamando a la función y guardando el DataFrame resultante en una nueva variable
df = eliminar_aerolineas(df, otros_transportes)

### Mapeo de las ciudades con el precio medio más barato

En esta sección se extrae una lista de los top 10 destinos con un precio medio de ticket más económico, además de graficar una lista de hasta 50 posibles sitios/lugares de interés para visitar en cada una de esas ciudades.

In [ ]:
df_lugares = df.groupby("ciudad_destino")['precio'].mean().sort_values()[:10].reset_index()

In [ ]:
df_lugares

In [ ]:
lista_ciudades = df_lugares["ciudad_destino"].unique()

data_lugares = []

lugares_columns = ['ciudad', 'name', 'fsq_id', 'category', 'latitude', 'longitude', 'country', 'full_location', 'postcode', 'closed_bucket']

In [ ]:
# Credenciales API KEY para API de FOURSQUARE

print(load_dotenv())

API_KEY = os.getenv('FOURSQUARE_API_KEY')

API_KEY

In [ ]:
for ciudad in lista_ciudades:

    # Base del Endpoint
    search_url = "https://api.foursquare.com/v3/places/search"

    url_params = {"categories"    : "16000",#Este id de categoría corresponde a Landmarks and Outdoors, para sacar posibles sitios de interés qué visitar
                  "near"       : ciudad,
                  "limit"    : 50}

    headers = {"accept"       : "application/json", 
               "Authorization": API_KEY}
    try:
        # Como ahora usamos una variable para los parámetros, debemos agregarlos con el parámetro "params"
        response = requests.get(url = search_url, params = url_params, headers = headers)

        print(f"response: {response.status_code}")
        print(f"endpoint: {response.url}")

        lugares = response.json()['results']

        sleep(2)

        counter = 0

        for lugar in lugares:

            category = lugar['categories'][0]['name'] if len(lugar['categories']) else None
            fsq_id = lugar['fsq_id']
            latitude = lugar['geocodes']['main']['latitude']
            longitude = lugar['geocodes']['main']['longitude']
            full_location = lugar['location']['formatted_address']
            country = lugar['location']['country']
            postcode = lugar['location'].get('postcode')
            closed_bucket = lugar['closed_bucket']
            name = lugar['name']

            data_lugares.append([ciudad, name, fsq_id, category, latitude, longitude, country, full_location, postcode, closed_bucket])
            counter += 1

        print(f'Extraídos {counter} lugares de {ciudad}')
        
    except:
        continue

In [ ]:
df_lugares = pd.DataFrame(data_lugares, columns = lugares_columns)
df_lugares.info()

In [ ]:
centro = (df_lugares['latitude'].mean(), df_lugares['longitude'].mean())

mapa = folium.Map(location = centro, zoom_start = 5)

sitios = folium.map.FeatureGroup()

for idx, row in df_lugares.iterrows():

    lat = row["latitude"]
    lng = row["longitude"]
    label = row["name"]
    category = row["category"]
    
    sitios.add_child(folium.Marker(location = [lat, lng],
                                    popup    = f"{label}, {category}"))
    
mapa.add_child(sitios)

mapa

#### ¿Cuántos destinos obtengo en cada página web en cada una de las fechas establecidas? ¿Cuántos vuelos por destino obtengo?

In [ ]:
#Para contestar a estas preguntas realizaremos una agrupación por fecha de inicio de cada vuelo y el destino.
#Así obtendremos el número de destinos únicos por cada vuelo. Así como el número de vuelos por cada destino.

destinos_por_fecha = df.groupby(["pagina_web", "fecha_inicio"])["destino"].nunique().reset_index()
vuelos_por_destino = df.groupby(["pagina_web", "ciudad_destino"]).size().reset_index(name = "num_vuelos")


In [ ]:
destinos_por_fecha

In [ ]:
vuelos_por_destino

In [ ]:
#Ordenamos las ciudades alfabéticamente
vuelos_por_destino_sorted = vuelos_por_destino.sort_values(by="ciudad_destino")

In [ ]:
#Representamos estos cálculos mediante un gráfico de barras para destinos por fecha y página web
fig_destinos = px.bar(destinos_por_fecha, x = "fecha_inicio", y = "destino", color = "pagina_web",
                      title = "Número de destinos por fecha",
                      labels = {"destino": "Número de destinos únicos"},
                      category_orders = {"pagina_web": ["kayak", "edreams"]})
fig_destinos.show()

#Y otro para vuelos por destino
fig_vuelos = px.bar(vuelos_por_destino, x = "ciudad_destino", y = "num_vuelos", color = "pagina_web",
                    title = "Número de vuelos por destino",
                    labels = {"num_vuelos": "Número de vuelos"},
                    category_orders = {"pagina_web": ["kayak", "edreams"]})
fig_vuelos.show()


In [ ]:
#Luego ya concatenamos los resultados y especificamos keys como: kayak y edreams.
#Así podriamos generar tablas para que se pueden extraer o analizarlas.
resumen_destinos = pd.concat([destinos_por_fecha[destinos_por_fecha['pagina_web'] == 'kayak'], 
                              destinos_por_fecha[destinos_por_fecha['pagina_web'] == 'edreams']], 
                             keys=["kayak", "edreams"])

resumen_vuelos = pd.concat([vuelos_por_destino[vuelos_por_destino['pagina_web'] == 'kayak'], 
                            vuelos_por_destino[vuelos_por_destino['pagina_web'] == 'edreams']], 
                           keys=["kayak", "edreams"])


En la gráfica en la que observamos el número de destinos por fecha, vemos que hemos obtenido una gran contidad de datos de datos de kayak en comparación a edreams. 

In [ ]:
df["pagina_web"].value_counts()

In [ ]:
resumen_destinos

In [ ]:
resumen_vuelos

Si nos interesa ver las aerolíneas que operan para cada destino, podríamos extraer información del resumen vuelo.
Además, aquí representamos el número de aerolíneas que operan en los vuelos que nos ofrece cada página web.

In [ ]:
#Con un boxplot podríamos visualizar si existen diferencias entre el número de aerolíneas que operan en los vuelos de los destinos
#que nos ofrece cada página web.

fig = px.box(resumen_vuelos, x = "pagina_web", y = "num_vuelos", color = "pagina_web",
             category_orders = {"pagina_web": ["kayak", "edreams"]})

# Actualizar las etiquetas del eje x y el título
fig.update_xaxes(tickvals = [0, 1], ticktext = ["Kayak", "eDreams"], title_text = "Página Web")
fig.update_yaxes(title_text = "Número de aerolíneas")
fig.update_layout(title_text = "Número de aerolíneas por destino")

# Mostrar el gráfico
fig.show()


### De todos estos vuelos: ¿cuál es el porcentaje de vuelos que opera con más de una aerolínea?

In [ ]:
#Para poder responder a esta pregunta se define la siguiente función en la que primero determinamos
#es los vuelos que tengan más de una aerolinea. Luego calculamos el porcentaje.
#Finalmente lo apliamos a los destinos filtrados pero en función de lo que nos ofrezca kayak o edreams.
#

def calcular_porcentaje_vuelos_mas_de_una_aerolinea(df, pagina_web):
    
    vuelos_mas_de_una_aerolinea = df[(df["num_aerolineas"] > 1) & (df["pagina_web"] == pagina_web)]
    porcentaje_vuelos_mas_de_una_aerolinea = (len(vuelos_mas_de_una_aerolinea) / len(df[df["pagina_web"] == pagina_web])) * 100
    
    return porcentaje_vuelos_mas_de_una_aerolinea

porcentaje_kayak = calcular_porcentaje_vuelos_mas_de_una_aerolinea(df, "kayak")
porcentaje_edreams = calcular_porcentaje_vuelos_mas_de_una_aerolinea(df, "edreams")


In [ ]:
#Convertimos los datos en DataFrame para poder representarlos en un gráfico de pastel.
data = {"Porcentaje": [porcentaje_kayak, porcentaje_edreams],
        "Pagina Web": ["Kayak", "eDreams"]}

df_porcentajes = pd.DataFrame(data)


fig = px.pie(df_porcentajes, names = "Pagina Web", values = "Porcentaje", title = "Porcentaje de vuelos con más de una aerolínea",
             labels = {"Porcentaje": "Porcentaje de Vuelos (%)"},
             color_discrete_sequence = px.colors.qualitative.Set1)

fig.show()


A lo mejor nos interesa que el vuelo sea gestionado por una sola aerolínea en caso de que haya retrasos o se deba pedir indemnizaciones. O bien, nos gusta el servicio de una aerolínea en concreto.

### De los distintos destinos que me ofrecen en kayak y eDreams: ¿cuál de ellos me ofrecen destinos más baratos? entre un rango de 100-250 euros.

In [ ]:
#Primero vamos a filtrar por el rango de precio que hemos definido.
def filtrar_destinos_por_precio(df, precio_min, precio_max):
    destinos_filtrados = df[(df["precio"] >= precio_min) & (df["precio"] <= precio_max)]
    
    return destinos_filtrados


In [ ]:
destinos_filtrados = filtrar_destinos_por_precio(df, 100, 250)

In [ ]:
destinos_filtrados['aerolineas'].unique()

In [ ]:
destinos_filtrados.info()

Si volvemos a agrupar el destino por fecha y los vuelos por destino una vez hemos definido un df con destinos filtrados en el rango de precio de 100-250€ vemos que el número de destino por cada fecha ha disminuído.

In [ ]:
destinos_por_fecha = destinos_filtrados.groupby(["pagina_web", "fecha_inicio"])["destino"].nunique().reset_index()
vuelos_por_destino = destinos_filtrados.groupby(["pagina_web", "fecha_inicio", "ciudad_destino"]).size().reset_index(name = "num_vuelos")

In [ ]:
#Ordenamos las ciudades alfabéticamente
vuelos_por_destino_sorted = vuelos_por_destino.sort_values(by="ciudad_destino")

In [ ]:
#Representamos estos cálculos mediante un gráfico de barras para destinos por fecha y página web
fig_destinos = px.bar(destinos_por_fecha, x = "fecha_inicio", y = "destino", color = "pagina_web",
                      title = "Número de destinos por fecha",
                      labels = {"destino": "Número de destinos únicos"},
                      category_orders = {"pagina_web": ["kayak", "edreams"]})
fig_destinos.show()

#Y otro para vuelos por destino
fig_vuelos = px.bar(vuelos_por_destino, x = "ciudad_destino", y = "num_vuelos", color = "pagina_web",
                    title = "Número de vuelos por pestino",
                    labels = {"num_vuelos": "Número de vuelos"},
                    category_orders = {"pagina_web": ["kayak", "edreams"]})
fig_vuelos.show()

In [ ]:
#Primero debemos calcular la duración total del vuelo (ida y vuelta)
destinos_filtrados["duracion_total"] = destinos_filtrados["duracion_ida"] + destinos_filtrados["duracion_vuelta"]


# Antes filtraremos
Q1, Q3 = np.percentile(destinos_filtrados['duracion_total'], [25, 75])
IQR = Q3 - Q1
techo = Q3 + 1.5 * IQR
piso = Q1 - 1.5 * IQR

In [ ]:
destinos_filtrados_2 = destinos_filtrados[destinos_filtrados['duracion_total'].between(piso, techo)].copy()

In [ ]:
fig = px.box(destinos_filtrados_2, x = "pagina_web", y = "duracion_total", color = "pagina_web",
             category_orders = {"pagina_web": ["kayak", "edreams"]})

fig.update_xaxes(tickvals = [0, 1], ticktext = ["Kayak", "eDreams"], title_text = "Página Web")
fig.update_yaxes(title_text = "Duración total (minutos)")
fig.update_layout(title_text = "Distribución de la duración total de vuelos (Ida-Vuelta)")

fig.show()

### Evaluaremos los horarios de ida y vuelta que nos ofrece kayak y edreams:

In [ ]:
# categorizaría las horas de salida y de vuelta para analizar de esos precios: 
#¿cuáles salen a primera hora de la mañana? Ya que nos gustaría aprovechar lo máximo. Lo ideal sería que salga a primera hora de la mañana, no?
#de los vuelos de vuelta, ¿cuáles salen a última hora? Nos gustaría volver a última hora para aprovechar el último día o por la tarde,
#así podemos llegar con calma, deshacer la maleta y descansa antes de volver a la rutina?

# Función para categorizar los horarios en 'Mañana' o 'Tarde'
def categorizar_horario(hora):
    if hora >= pd.to_datetime('5:00:00', format='%H:%M:%S').time() and hora <= pd.to_datetime('14:00:00', format='%H:%M:%S').time():
        return 'Mañana'
    elif hora >= pd.to_datetime('14:00:01', format='%H:%M:%S').time() and hora <= pd.to_datetime('23:59:59', format='%H:%M:%S').time():
        return 'Tarde'
    elif hora >= pd.to_datetime('00:00:00', format='%H:%M:%S').time() and hora <= pd.to_datetime('04:59:59', format='%H:%M:%S').time():
        return 'Tarde'
    else:
        return 'Otro'

#Aplicamos la función de categorización a las columnas de inicio y fin (ida y vuelta)
df[['categoria_inicio_ida', 'categoria_inicio_vuelta']] = df[['inicio_ida', 'inicio_vuelta']].applymap(lambda x: categorizar_horario(x))

# También lo aplicaré al df de destinos_filtrados
destinos_filtrados['categoria_inicio_ida'] = destinos_filtrados['inicio_ida'].apply(categorizar_horario)
destinos_filtrados['categoria_inicio_vuelta'] = destinos_filtrados['fin_vuelta'].apply(categorizar_horario)

In [ ]:
df.head(2)

In [ ]:
destinos_filtrados.head(2)

In [ ]:
#Tras la categorización quiero saber qué página web me ofrece vuelos de inicio_ida de Mañana y vuelos de inicio_vuelta de Tarde
#Para eso agrupo mis datos filtrados por página web, la categoría de horas y contamos el número de vuelos.

vuelos_manana_ida = destinos_filtrados[destinos_filtrados["categoria_inicio_ida"] == "Mañana"].groupby("pagina_web")["ciudad_destino"].unique()
vuelos_tarde_vuelta = destinos_filtrados[destinos_filtrados["categoria_inicio_vuelta"] == "Tarde"].groupby("pagina_web")["ciudad_destino"].unique()

In [ ]:
list(vuelos_manana_ida)

Observamos que el número de vuelos que me ofrecen salir por la mañana y volver por la tarde es similar para cada página web. 

In [ ]:
# Primero creamos un df para los resultados
data = {
    'pagina_web': ['kayak', 'edreams'],
    'vuelos_manana_ida': [len(vuelos_manana_ida['kayak']), len(vuelos_manana_ida['edreams'])],
    'vuelos_tarde_vuelta': [len(vuelos_tarde_vuelta['kayak']), len(vuelos_tarde_vuelta['edreams'])]
}

df_resultados = pd.DataFrame(data)

# Generamos el gráfico de barras apiladas para visualizar la cantidad de vuelos que me ofrecen una ida de mañana y vuelta de tarde
fig = px.bar(df_resultados, x = "pagina_web", y = ["vuelos_manana_ida", "vuelos_tarde_vuelta"],
             title = "Número de destinos únicos por categoría horaria",
             labels = {"value": "Número de destinos únicos"},
             color_discrete_sequence=["lightblue", "orange"])

fig.update_layout(barmode="stack")
fig.show()


### Evaluaremos las condiciones de escalas, equipaje, las aerolíneas que operan:

In [ ]:
def analizar_condiciones_companias(df, pagina_web):
    df_pagina_web = df[df["pagina_web"] == pagina_web]
    
    resultados = {}
    
    # Equipaje de mano permitido
    vuelos_con_equipaje_mano = df_pagina_web[df_pagina_web["equipaje_mano"] > 0].shape[0]
    resultados["vuelos_con_equipaje_mano"] = vuelos_con_equipaje_mano
    
    # Equipaje de bodega permitido
    vuelos_con_equipaje_bodega = df_pagina_web[df_pagina_web["equipaje_bodega"] > 0].shape[0]
    resultados["vuelos_con_equipaje_bodega"] = vuelos_con_equipaje_bodega
    
    # Número de aerolíneas únicas que operan
    aerolineas_unicas = df_pagina_web["aerolineas"].explode().unique()
    cantidad_aerolineas_unicas = len(aerolineas_unicas)
    resultados["cantidad_aerolineas_unicas"] = cantidad_aerolineas_unicas
    
    # Número de escalas en los vuelos de ida
    numero_escalas_ida = df_pagina_web["escala_ida"].apply(lambda x: len(x) if isinstance(x, (list, str)) else 0).sum()
    resultados["numero_escalas_ida"] = numero_escalas_ida
    
    # Número de escalas en los vuelos de vuelta
    numero_escalas_vuelta = df_pagina_web["escala_vuelta"].apply(lambda x: len(x) if isinstance(x, (list, str)) else 0).sum()
    resultados["numero_escalas_vuelta"] = numero_escalas_vuelta
    
    return resultados

#Aplicamos la función al df de destinos_filtrados y la página web específica
resultados_condiciones_kayak = analizar_condiciones_companias(destinos_filtrados, "kayak")
resultados_condiciones_edreams = analizar_condiciones_companias(destinos_filtrados, "edreams")

In [ ]:
resultados_condiciones_kayak

In [ ]:
resultados_condiciones_edreams

In [ ]:
#Creamos un DataFrame con los resultados específicos para cada página web
resultados_df = pd.DataFrame({
    "Categorias": ["Vuelos con equipaje de mano", "Vuelos con equipaje de bodega",
                  "Cantidad de aerolíneas únicas"],
    "Kayak": [
        resultados_condiciones_kayak["vuelos_con_equipaje_mano"],
        resultados_condiciones_kayak["vuelos_con_equipaje_bodega"],
        resultados_condiciones_kayak["cantidad_aerolineas_unicas"]
    ],
    "eDreams": [
        resultados_condiciones_edreams["vuelos_con_equipaje_mano"],
        resultados_condiciones_edreams["vuelos_con_equipaje_bodega"],
        resultados_condiciones_edreams["cantidad_aerolineas_unicas"]
    ]
})

In [ ]:
fig = px.scatter(resultados_df, x = "Categorias", y = ["Kayak", "eDreams"],
                 title = "Comparación de condiciones de vuelo: Kayak vs eDreams",
                 labels = {"value": "Cantidad"},
                 color_discrete_sequence = ["blue", "orange"])

fig.show()


In [ ]:
# Obtener la lista de aerolíneas únicas que operan en los vuelos
aerolineas_unicas = destinos_filtrados['aerolineas'].explode().unique()
print(f'Aerolíneas que operan en los vuelos: {aerolineas_unicas}')
